In [1]:
import pyvisa
import time
import schedule
import pandas as pd
import serial.tools.list_ports
import os

Port_Arduino = 'COM10'

# Destination folder and file path
folder_path = "C:\\Users\\kary\\Documents\\9_data\\IEAP\\"
file_path = os.path.join(folder_path, "voltage_data.csv")

#Create Resource manager instance
rm = pyvisa.ResourceManager()
#Check the VISA device
rm.list_resources()

('USB0::0x05E6::0x6500::04537333::INSTR', 'ASRL10::INSTR')

In [2]:
# Start the communication by VISA identifying resource name
dmm = rm.open_resource('USB0::0x05E6::0x6500::04537333::INSTR')

#Checking the machine name
print(dmm.query('*IDN?'))

KEITHLEY INSTRUMENTS,MODEL DMM6500,04537333,1.7.11b



In [3]:
def add_row(df, row_data):
    """データフレームに新しい行を追加"""
    df_length = len(df)
    df.loc[df_length] = row_data
    return df

In [4]:
def df_add_read_data(df, dmm, total_read_time, sample_period):
    # DMMの設定
    buffer_name = "buffer"
    buffer_size = 10000
    units = "VOLT"
    display_digits = 3

    # バッファを作成
    dmm.write(f':TRAC:MAKE "{buffer_name}", {buffer_size}, WRITable')
    
    # データフォーマットを設定
    dmm.write(f':TRAC:WRIT:FORM "{buffer_name}", {units}, {display_digits}')
    
    # 測定時間の初期化
    start_time = time.perf_counter()
    end_time = time.perf_counter()

    while end_time - start_time < total_read_time:
        each_start_time = time.perf_counter()
        
        # 測定データをバッファに書き込む
        dmm.write(':INIT')
        dmm.write(f':TRAC:WRIT:READ "{buffer_name}", 1')
        
        # バッファからデータを読み取る
        try:
            read_data = float(dmm.query(':TRAC:DATA? 1, 1, "{buffer_name}", read'))
        except pyvisa.errors.VisaIOError as e:
            print(f"Error reading data: {e}")
            break
        each_end_time = time.perf_counter()
        
        # サンプル周期になるまで待機
        while each_end_time - each_start_time <= sample_period:
            each_end_time = time.perf_counter()

        df = add_row(df, [each_end_time - start_time, read_data])
        end_time = time.perf_counter()

    return df

In [5]:
columns = ["Time[s]","Measured_Voltage[V]"]
df_voltage = pd.DataFrame(columns=columns)

position = 0
read_time = 1 # sec
freq = 0.001
df_voltage = df_add_read_data(df_voltage,dmm,read_time,freq)
df_voltage

Error reading data: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.


,Time[s],Measured_Voltage[V]


In [9]:
# バッファを作成する
buffer_name = "buffer4"
buffer_size = 1000
dmm.write(f':TRAC:MAKE "{buffer_name}", {buffer_size}, WRITable')

# データのフォーマットを設定する
dmm.write(f':TRAC:WRIT:FORM "{buffer_name}", VOLT, 3')

# データをバッファに書き込む
dmm.write(f':TRAC:WRIT:READ "{buffer_name}", 1')

# バッファからデータを読み取る
read_data = dmm.query(f':TRAC:DATA? 1, 1, "{buffer_name}", read')
print(read_data)


1.000000E+00

